In [1]:
import trade_model as td
dm = td.DataModel("../xcels", ["master0.csv", "master1.csv"])
dm.read()

In [20]:
symbol = "فولاد"
# df = dm.get(symbol, "2017-05-19", "2017-09-03")
df = dm.get(symbol, "1394-4-23", "1394-6-18")

In [62]:
# %%writefile trade_model.py

import pandas as pd
from khayyam import JalaliDate, JalaliDatetime

def add_diff_min_max(df): 
    df["diff_min_max"] = (df['max']-df['min'])*100/(df['min'])

def add_diff_ending(df):
    df["diff_open"] = (df['lastday']-df['ending'])*100/(df['lastday'])

def add_adjust_scale(df_symbol):
    lastdays = df_symbol["lastday"].copy().drop(df_symbol.index[0])
    endings = df_symbol["ending"].copy().drop(df_symbol.index[-1])
    endings.index = lastdays.index
    scale = lastdays/endings
    scale[df_symbol.index[0]] = 1
    df_symbol.loc[:, "adj_scale"] = scale
    
def add_adjust(df):
    adj = df.loc[df["adj_scale"] < 1].index
    df["adj_open"] = df["open"]
    df["adj_close"] = df["close"]
    df["adj_ending"] = df["ending"]
    df["adj_min"] = df["min"]
    df["adj_max"] = df["max"]
    adj_headers = ["adj_min", "adj_max", "adj_close", "adj_open", "adj_ending"]
    for date in adj:
        scale = df.loc[date, "adj_scale"]
        df.loc[df.index[0]:date, adj_headers] = df.loc[df.index[0]:date, adj_headers].transform(lambda x: x * scale)

def add_log_adj(df):
    adj = df.loc[df["adj_scale"] < 1].index
    df["log_adj_open"] = df["open"]
    df["log_adj_close"] = df["close"]
    df["log_adj_ending"] = df["ending"]
    df["log_adj_min"] = df["min"]
    df["log_adj_max"] = df["max"]
    adj_headers = ["log_adj_min", "log_adj_max", "log_adj_close", "log_adj_open", "log_adj_ending"]
    for date in adj:
        scale = df.loc[date, "adj_scale"]
        df.loc[df.index[0]:date, adj_headers] = df.loc[df.index[0]:date, adj_headers].transform(lambda x: x * scale)

class DataModel:
    def __init__(self,data_location, file_names=[]):  
        self.data_location = data_location;
        self.file_names = file_names;
    
    def __read_csv(self, file_name):
        return pd.read_csv(f'{self.data_location}/{file_name}', sep=',',header=[0], parse_dates=["date"])

#     TODO: use pystore instead
    def read(self):
        dfs = []
        for name in self.file_names:
            dfs.append(self.__read_csv(name))
        self.df = pd.concat(dfs, ignore_index=True)
        add_diff_min_max(self.df)
        add_diff_ending(self.df)
        self.df = self.df.set_index('date')
#         print("hi")
#         self.df = self.df.groupby("symbol").apply(add_adjust_scale)
#         self.allSymbols = self.df.symbol.tolist()
#         self.symbols = list(set(self.df.symbol))[1:]
#         for symbol in self.symbols:
#         counts = Counter(self.allSymbols)
#         testSymbols = []
#         tmpSymbols = []
#         for symbol in symbols:
#             if counts[symbol] > RECORD_THRESHOLD:
#                 tmpSymbols.append(symbol)
#         for i in range(TESTCASE_NUMBER):
#             ran = random.randint(0, len(tmpSymbols)-1)
#             testSymbols.append(tmpSymbols[ran])
#             tmpSymbols.remove(tmpSymbols[ran])
#         print("test symbol", len(testSymbols))


    def get(self, symbol, start="", end=""):
        
        if start == "":
            start = self.df.index[0]
        else:
            s_date = start.split("-")
            start = JalaliDate(s_date[0], s_date[1], s_date[2]).todate()
        if end == "":
            end = self.df.index[-1]
        else:
            e_date = end.split("-")
            end = JalaliDate(e_date[0], e_date[1], e_date[2]).todate()
        tmpdf = self.df.loc[start:end]
        return tmpdf.loc[tmpdf["symbol"]==symbol].copy()


SyntaxError: invalid syntax (<ipython-input-62-37e350791bff>, line 32)

In [2]:
dm = DataModel("../xcels", ["master0.csv", "master1.csv"])
dm.read()

In [61]:
symbol = "خساپا"
# dm.get(symbol, "1381-12-27", "1382-01-20")

df = dm.get(symbol,"1398-4-28", "1398-5-2")
# df = dm.get(symbol)
add_adjust_scale(df)
abnormal = df[df["adj_scale"] < 1]
for a in abnormal.index:
    print(JalaliDate(a), abnormal.loc[a, "adj_scale"])
add_adjust(df)

df.loc[:, ["close", "open", "ending", "lastday"]]

,close,open,ending,lastday
date,,,,
2019-07-20,1499.0,1510.0,1565.0,1526.0
2019-07-24,1520.0,1545.0,1534.0,1565.0
